In [5]:
import lightgbm as lgb
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc
import time
from sklearn.metrics import roc_auc_score

In [2]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [3]:
train_csr = load_csr('../../data/features/xgb/train_csr_2500')
predict_csr = load_csr('../../data/features/xgb/predict_csr_f620_b70_col17k')
train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')
predicted_file = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')
instance_train = pd.read_pickle('../../data/features/xgb/train_noclean_bytime.pkl')[['instance_id']].copy()
instance_test = predicted_file[['instance_id']].copy()

data load
indices load
indptr load
data load
indices load
indptr load


# load LibFM

In [4]:
fm_train = pd.read_pickle(FILE.X_fe_train_libfm.value)
fm_test = pd.read_pickle(FILE.X_fe_test_libfm.value)
fm_train = instance_train.merge(fm_train,how='inner',on='instance_id')
fm_train.drop('instance_id',axis=1,inplace=True)
print(fm_train.shape)
fm_test = instance_test.merge(fm_test,how='inner',on='instance_id')
fm_test.drop('instance_id',axis=1,inplace=True)
print(fm_test.shape)

(2992639, 27)
(80276, 27)


In [6]:
fm_train.columns

Index(['uid1+uraw1+itemc1_matrix_factorization',
       'uid1+uraw1+itemc2_matrix_factorization',
       'uid1+uraw1+itemc3_matrix_factorization',
       'uid1+uraw2+itemc1_matrix_factorization',
       'uid1+uraw2+itemc2_matrix_factorization',
       'uid1+uraw2+itemc3_matrix_factorization',
       'uid1+uraw3+itemc1_matrix_factorization',
       'uid1+uraw3+itemc2_matrix_factorization',
       'uid1+uraw3+itemc3_matrix_factorization',
       'uid2+uraw1+itemc1_matrix_factorization',
       'uid2+uraw1+itemc2_matrix_factorization',
       'uid2+uraw1+itemc3_matrix_factorization',
       'uid2+uraw2+itemc1_matrix_factorization',
       'uid2+uraw2+itemc2_matrix_factorization',
       'uid2+uraw2+itemc3_matrix_factorization',
       'uid2+uraw3+itemc1_matrix_factorization',
       'uid2+uraw3+itemc2_matrix_factorization',
       'uid2+uraw3+itemc3_matrix_factorization',
       'uid3+uraw1+itemc1_matrix_factorization',
       'uid3+uraw1+itemc2_matrix_factorization',
       'uid3+uraw1+i

In [7]:
roc_auc_score(train_y,fm_train['uid1+uraw1+itemc1_matrix_factorization'].values)

0.7591925513693333

In [8]:
col_list = []
roc_list = []
for col in tqdm(fm_train.columns):
    score = roc_auc_score(train_y,fm_train[col].values)
    col_list.append(col)
    roc_list.append(score)

100%|██████████| 27/27 [00:35<00:00,  1.32s/it]


In [9]:
df_score = pd.DataFrame({'score':roc_list,'col':col_list})

In [12]:
df_score.sort_values('score')

,score,col
12,0.743175,uid2+uraw2+itemc1_matrix_factorization
13,0.743376,uid2+uraw2+itemc2_matrix_factorization
10,0.743385,uid2+uraw1+itemc2_matrix_factorization
16,0.743533,uid2+uraw3+itemc2_matrix_factorization
15,0.743750,uid2+uraw3+itemc1_matrix_factorization
9,0.743866,uid2+uraw1+itemc1_matrix_factorization
14,0.744024,uid2+uraw2+itemc3_matrix_factorization
11,0.747078,uid2+uraw1+itemc3_matrix_factorization
17,0.747400,uid2+uraw3+itemc3_matrix_factorization
19,0.758432,uid3+uraw1+itemc2_matrix_factorization


# Put to submission

In [15]:
cv_loss = 0.5
n_fold = 5
colnum = 1

predict_result = instance_test.copy()

for col in fm_train.columns:
    predict_result = instance_test.copy()
    oof = fm_train[col].values
    predict_result['predicted_score']= fm_test[col].values
    now = str(np.random.randint(1000000))
    
    predict_result[['instance_id', 'predicted_score']].to_csv( "../../data/features/agg/submission/%s_libFm_n%d_b%d_1h_col%d.csv" % (now, n_fold, 70, colnum), index=False)
    np.save('../../data/features/agg/submission/oof_%s_%.5f'%(now, cv_loss), oof) 